In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import xlwings as xw
import time

In [3]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('disable-notifications')

# Medica
# driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_driver_path = "../chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver_path, options = chrome_options)
url = 'https://www.medica.de/vis/v1/de/search?oid=80396&lang=1&_query=&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03&f_type=profile'
driver.get(url)
driver.maximize_window()

# Shadow Root, falls Programm es nicht findet, selber klicken
shadow_root = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root").shadow_root
sel = '#uc-center-container > div.sc-kdBSHD.gsTflx > div > div.sc-dtBdUo.fzvttm > div > button.sc-dcJsrY.eyWWqo'
accept = WebDriverWait(shadow_root, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,sel))) 
accept.click()

AttributeError: 'WebElement' object has no attribute 'shadow_root'

In [6]:
# Function to scroll down the page to load more content
def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Find the parent div element using its ID
parent_div_id = 'vis-search-scroll-area'
parent_div = driver.find_element(By.ID, parent_div_id)

unique_elements = []

# Maximum number of iterations to scroll down
max_scroll_iterations = 100

# Scroll down and collect div elements until the maximum number of iterations or no new elements are found
for _ in range(max_scroll_iterations):
    prev_elements_count = len(unique_elements)
    
    # Scroll down the page to load more content
    scroll_down()
    time.sleep(1)

    # Wait for some time to let the new content load
    WebDriverWait(driver, 30).until(lambda d: len(d.find_elements(By.XPATH, './/div')) > prev_elements_count)
    
    child_div_class_name = 'searchresult-item'
    child_div_elements = parent_div.find_elements(By.CLASS_NAME, child_div_class_name)

    new_elements_found = False
    
    for div_element in child_div_elements:
        element_text = div_element.text
        # Check if the element's text is not already in the set before adding it to the list
        if element_text not in unique_elements:
            # Add the element's text to the set
            unique_elements.append(element_text)
            new_elements_found = True

    # If no new elements are found in this iteration, break the loop
    if not new_elements_found:
        break

In [7]:
unique_elements_set = list(unique_elements)

name = [None]*len(unique_elements_set)
name_search = [None]*len(unique_elements_set)
country = [None]*len(unique_elements_set)
hall = [None]*len(unique_elements_set)

for i in range(len(unique_elements_set)):
    if 'Aussteller\n' in unique_elements_set[i]:
        index_aussteller = unique_elements_set[i].index('Aussteller\n') + len('Aussteller\n')
        unique_elements_set[i] = unique_elements_set[i][index_aussteller:]
        while True:
            if 'Aussteller\n' in unique_elements_set[i]:
                index_aussteller = unique_elements_set[i].index('Aussteller\n') + len('Aussteller\n')
                unique_elements_set[i] = unique_elements_set[i][index_aussteller:]
            else:
                break        
        name[i] = unique_elements_set[i].split('\n')[0].strip()
        name_search[i] = unique_elements_set[i].split('\n')[0].strip()
        hall[i] = unique_elements_set[i].split('\n')[-1].strip()
        land = unique_elements_set[i].split('\n')[1].split(',')[-1].strip()
        if land == 'Republik':
            land = unique_elements_set[i].split('\n')[1].split(',')[-2].strip()
        country[i] = land
    else:
        name[i] = unique_elements_set[i].split('\n')[-2].split(':')[-1].strip()
        name_search[i] = unique_elements_set[i].split('\n')[0].strip()
        hall[i] = unique_elements_set[i].split('\n')[-1].strip()
        country[i] = ''
    # Überschreiben von name_search, falls es sich um Produkt handelt
    if 'Produkt\n' in unique_elements_set[i]:
        name_search[i] = unique_elements_set[i].split('Produkt\n')[1].split('\n')[0].strip()

In [8]:
# country name ger to eng
country_dict = {'Frankreich': 'France',
               'Korea': 'South Korea',
               'Italien': 'Italy',
               'Indien': 'India',
               'Deutschland': 'Germany',
               'Polen': 'Poland',
               'Belgien': 'Belgium',
               'Kanada': 'Canada',
               'Schweiz': 'Switzerland',
               'Österreich': 'Austria',
               'Spanien': 'Spain',
               'Türkei': 'Turkey',
               'Großbritannien und Nordirland': 'UK',
               'Luxemburg': 'Luxembourg',
               'Islamische Republik': 'Islamic Republic',
               'Dänemark': 'Denmark',
               'Niederlande': 'Netherlands',
               'Schweden': 'Sweden',
               'Ägypten': 'Egypt',
               'Tunesien': 'Tunesia',
               'Finnland': 'Finland',
               'Litauen': 'Lithuania',
               'Estland': 'Estonia',
                'Kroatien': 'Croatia',
                'Ungarn': 'Hungary',
                'Griechenland': 'Greece',
                'Tschechien': 'Czech',
                'Australien': 'Australia',
                'Brasilien': 'Brazil',
                'Slowenien': 'Slovenia',
                'Norwegen': 'Norway'
               }

for i in range(len(country)):
    if country[i] in country_dict:
        country[i] = country_dict[country[i]]

In [9]:
# Dopplungen entfernen
drop_index = []
for i in range(len(name)-1):
    if (name[i] == name[i+1]) & (country[i] == country[i+1]) & (hall[i] == hall[i+1]):
        drop_index.append(i)
        
name = [name[i] for i in range(len(name)) if i not in drop_index]
name_search = [name_search[i] for i in range(len(name_search)) if i not in drop_index]
country = [country[i] for i in range(len(country)) if i not in drop_index]
hall = [hall[i] for i in range(len(hall)) if i not in drop_index]

# Excel ausfüllen
wb = xw.Book('2023 AACC Ausstellerliste_Vorlage.xlsx')
sheet = wb.sheets[0]

A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
A_col_range.value = [[value] for value in name]

B_col_range = sheet.range('B3:B{}'.format(3+len(country)))
B_col_range.value = [[value] for value in country]

In [19]:
url = []
url_booth = []

for n in name_search:
    # get xpath based on name
    target_text = n
    
    if "'" in target_text:
        target_text = target_text.split("'")[0]

    # Find the element using its text with an XPath query
    xpath_query = f"//*[contains(text(), '{target_text}')]"
    element = driver.find_element(By.XPATH, xpath_query)

    # Evaluate the XPath of the element using JavaScript execution
    element_xpath = driver.execute_script(
        "function getElementXPath(element) {"
        "    if (element && element.id) {"
        "        return '//' + element.tagName.toLowerCase() + '[@id=\"' + element.id + '\"]';"
        "    } else {"
        "        return getElementXPath(element.parentElement) + '/' + element.tagName.toLowerCase();"
        "    }"
        "}"
        "return getElementXPath(arguments[0]);",
        element
    )
    # Open the link in a new tab using CTRL+click
    action = ActionChains(driver)
    action.key_down(Keys.CONTROL).click(element).key_up(Keys.CONTROL).perform()

    # Switch to the newly opened tab
    driver.switch_to.window(driver.window_handles[1])

    # Append the current URL of the newly opened tab to the url list
    url.append(driver.current_url)
    
    # Scroll up by sending the "PAGE_UP" key using Keys class
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.PAGE_UP)
    time.sleep(1)
    
    try:
        booth = driver.find_element(By.XPATH, '//*[@id="finder-profile"]/div/div/section/div/div/div[1]/div[2]/ul/li/a/div/button/div/span[2]')

        # Open the link in a new tab using CTRL+click
        action = ActionChains(driver)
        action.key_down(Keys.CONTROL).click(booth).key_up(Keys.CONTROL).perform()      

        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(1)
        url_booth.append(driver.current_url)
    
        # Close the newly opened tab
        driver.close()
        driver.switch_to.window(driver.window_handles[1])
    except:
        url_booth.append('')

    # Close the opened tab
    driver.close()
    # Switch back to the original tab
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)


In [23]:
# add hyperlinks
row_start = 558
for i in range(len(url)):
    row = 3+i+row_start
    cell_row, cell_column = row, 9
    cell_address = (cell_row, cell_column)
    sheet.range(cell_address).add_hyperlink(url_booth[i], text_to_display=hall[row_start+i])
    
    cell_row, cell_column = row, 1
    cell_address = (cell_row, cell_column)
    sheet.range(cell_address).add_hyperlink(url[i], text_to_display=name[row_start+i])
    sheet.range(cell_address).api.Font.Bold = True
    sheet.range(cell_address).font.color = (0, 0, 0)
    sheet.range(cell_address).api.Font.Size = 14
    sheet.range(cell_address).api.Font.Name = 'Verdana'

In [22]:
url

['https://www.medica.de/vis/v1/de/exhprofiles/0cuqvBtGS5uULrFzhpUkVA?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/exhprofiles/tIiEEXa9QZGt7C8zkFCclw?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/exhprofiles/HuGvRF7oTDG2IbX4EXQ54Q?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/exhprofiles/lW0dnnYgQsmArclGOUa8gQ?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/exhprofiles/YqLWSXtCSpaDr8quk9VrtA?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/exhprofiles/r4JonErLTsqGwcNgPUrmlQ?oid=80396&lang=1&_query=&f_type=profile&f_event=medcom2023.MEDICA&f_area=1.01&f_area=1.03',
 'https://www.medica.de/vis/v1/de/